In [9]:
import nltk # natural language processing
from nltk.stem import PorterStemmer #stemming words
from nltk.tokenize import word_tokenize,sent_tokenize # tokenizing into words and sentences
from nltk.corpus import stopwords #stopwords in english
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))



['testData.tsv', 'sampleSubmission.csv', 'labeledTrainData.tsv', 'unlabeledTrainData.tsv']


In [10]:
lbl_train = pd.read_csv('../input/labeledTrainData.tsv',  sep= '\t', header = 0)
test_d =    pd.read_csv('../input/testData.tsv',  sep= '\t', header = 0)
lbl_train.size,test_d.size


(75000, 50000)

In [11]:
total=lbl_train['review'].size + test_d["review"].size
total    

50000

In [12]:
###########data preprocessing
from bs4 import BeautifulSoup
import re
def raw_rev(rev):
    r_tag = BeautifulSoup(rev).get_text()                  ###removing html tags
    reg = re.sub("[^a-z/A-Z ]"," ",r_tag)                  ###excluding special characters
    normal = reg.lower()                                   ##normalizing text
    token = word_tokenize(normal)                          #####tokenizng
    stop_words = stopwords.words("english")                ### considering stopwords in english
    fi = []                                                  ###assigned a list
    [fi.append(w) for w in token if w not in stop_words]   ###creating a list excluding stopwords
    t =" ".join(fi)                                   ####joining words bac
    return (t)

In [13]:
#############filtered text
clean_train = []
clean_test = []
for i in range (0,lbl_train["review"].size):
     clean_train.append(raw_rev(lbl_train["review"][i]))
for j in range(0,test_d["review"].size):
     clean_test.append(raw_rev(test_d["review"][j]))

In [62]:
from keras.preprocessing.text import one_hot
one_h_train = [one_hot(review,2000) for review in clean_train]
one_h_test = [one_hot(review,2000) for review in clean_test]
print(one_h_train[0]), print(one_h_test[0])

[825, 958, 82, 836, 650, 1034, 1957, 303, 1148, 599, 1733, 381, 1733, 1124, 461, 1882, 144, 1968, 1214, 506, 1281, 1998, 1022, 1215, 461, 465, 365, 225, 1828, 1838, 1124, 1523, 938, 1523, 696, 670, 1815, 958, 1903, 1164, 1182, 1976, 669, 1230, 836, 1480, 1029, 1230, 975, 16, 1704, 64, 178, 128, 1832, 1733, 1452, 850, 1033, 975, 1675, 442, 836, 875, 958, 632, 1115, 132, 737, 1380, 836, 1065, 1437, 209, 1188, 836, 1816, 737, 1829, 1467, 1816, 200, 1998, 51, 1852, 696, 670, 68, 1725, 1348, 299, 199, 411, 1822, 1949, 354, 291, 1746, 1417, 1822, 1534, 1592, 848, 836, 1089, 178, 733, 836, 1051, 1418, 1921, 354, 291, 133, 201, 95, 125, 1654, 1677, 64, 475, 1654, 461, 1309, 836, 1957, 1112, 1022, 304, 442, 836, 1677, 1694, 1063, 1028, 1909, 484, 1949, 975, 1850, 1938, 1366, 742, 1548, 701, 1750, 178, 1949, 1012, 1949, 632, 1391, 721, 1447, 1778, 1279, 1028, 1373, 1513, 842, 1729, 1838, 1309, 93, 1188, 125, 442, 836, 721, 810, 1013, 1860, 125, 975, 411, 1762, 1307, 167, 1704, 1672, 836, 982, 78

(None, None)

In [32]:
from keras.preprocessing import sequence, text
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

X_train = sequence.pad_sequences(one_h_train, maxlen=350)
X_test = sequence.pad_sequences(one_h_test, maxlen=350)
print(X_train.shape,X_test.shape)



(25000, 350) (25000, 350)


In [33]:
x_t= X_train[:20000]
x_l = lbl_train.sentiment[:20000]
valid_t= X_train[20000:]
valid_l =lbl_train.sentiment[20000:]


In [ ]:
x_l.shape

In [ ]:
####word cloud
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
wc= WordCloud(height = 500, width = 1000).generate("".join(fea))
plt.figure(figsize=(10,10))
plt.imshow(wc, interpolation= 'bilinear')
plt.axis("off")
plt.margins(x=0,y=0)
plt.show

In [53]:
import tensorflow as tf
from tensorflow import keras
from keras. layers import Dropout,Dense,Flatten
from keras.models import Sequential
vocab=10000
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab,400,input_length=350))
model.add(keras.layers.LSTM(400, return_sequences =True))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dense(10000,activation = "softmax"))
model.add(keras.layers.Dense(1,activation = "sigmoid"))




In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 350, 400)          4000000   
_________________________________________________________________
lstm_7 (LSTM)                (None, 350, 400)          1281600   
_________________________________________________________________
dropout_7 (Dropout)          (None, 350, 400)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 140000)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 140001    
Total params: 5,421,601
Trainable params: 5,421,601
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.compile(optimizer=tf.train.AdamOptimizer(), loss = 'binary_crossentropy', metrics= ['accuracy'])

In [56]:
###training
training= model.fit(X_train,lbl_train.sentiment, epochs=10, batch_size = 500,verbose = 1)

Epoch 1/10
25000/25000 [==============================] - 149s 6ms/step - loss: 0.6189 - acc: 0.6406
Epoch 2/10
25000/25000 [==============================] - 144s 6ms/step - loss: 0.3952 - acc: 0.8236
Epoch 3/10
25000/25000 [==============================] - 144s 6ms/step - loss: 0.3554 - acc: 0.8438
Epoch 4/10
25000/25000 [==============================] - 144s 6ms/step - loss: 0.3346 - acc: 0.8535
Epoch 5/10
25000/25000 [==============================] - 145s 6ms/step - loss: 0.3020 - acc: 0.8710
Epoch 6/10
25000/25000 [==============================] - 146s 6ms/step - loss: 0.2709 - acc: 0.8862
Epoch 7/10
25000/25000 [==============================] - 146s 6ms/step - loss: 0.2308 - acc: 0.9059
Epoch 8/10
25000/25000 [==============================] - 146s 6ms/step - loss: 0.1827 - acc: 0.9287
Epoch 9/10
25000/25000 [==============================] - 145s 6ms/step - loss: 0.1394 - acc: 0.9472
Epoch 10/10
25000/25000 [==============================] - 146s 6ms/step - loss: 0.1020 - a

In [57]:
wa = model.predict(X_test)
wa1 = wa>0.5

In [58]:
wa[0]
test_d["sentiment"] = test_d["id"].map(lambda r: 1 if int(r.strip('"').split("_")[1]) >= 5 else 0)
ans = test_d["sentiment"]

In [59]:
ans

0        1
1        0
2        0
3        0
4        1
5        1
6        0
7        0
8        0
9        1
10       0
11       1
12       0
13       1
14       0
15       0
16       0
17       0
18       1
19       0
20       0
21       0
22       1
23       0
24       1
25       0
26       0
27       0
28       0
29       0
        ..
24970    1
24971    1
24972    0
24973    0
24974    0
24975    0
24976    1
24977    1
24978    1
24979    1
24980    1
24981    1
24982    1
24983    1
24984    1
24985    0
24986    1
24987    1
24988    0
24989    0
24990    0
24991    0
24992    0
24993    1
24994    0
24995    1
24996    1
24997    0
24998    1
24999    1
Name: sentiment, Length: 25000, dtype: int64

In [60]:
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(wa1, ans)))
print('Confusion matrix:')
confusion_matrix(wa1, ans)

F1-score: 0.7793629860528314
Confusion matrix:


array([[9728, 2749],
       [2772, 9751]])

In [61]:
data = {"id":test_d["id"], "sentiment":ans}
submission = pd.DataFrame(data, columns =['id', 'sentiment'])
submission

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,1
5,2913_8,1
6,4396_1,0
7,395_2,0
8,10616_1,0
9,9074_9,1
